
# CAE-Transformer降阶模型——开尔文-亥姆霍兹不稳定性问题

## 概述

为有效降低使用CFD方法的设计成本和周期，近年来降阶模型得到了广泛的关注。对于复杂的可压缩流动，使用本征正交分解(POD)等线性方法进行流场降维，需要大量的模态才能保证流场重建的精度。已有研究表明，采用非线性降维方法能够有效减少所需模态数。卷积自编码器(CAE)是一种由编码器和解码器组成的神经网络，能够实现数据降维和重构，可看作是POD方法的非线性拓展。采用CAE进行流场数据的非线性降维，同时使用Transformer进行流场状态的时间演化。针对非定常可压缩流动，CAE-Transformer降阶模型能够在使用较少自由变量数的前提下获得较高的重构和预测精度。

## 模型架构

CAE-Transformer降阶模型采用CAE网络对流场进行降维，提取流动数据的特征，将其压缩到编码器的隐藏空间中，再用Transformer网络通过注意力机制对隐藏空间的自由变量进行时间演化，得到流动其他时刻的自由变量，再通过CAE网络的解码器将演化的自由变量进行解码，重建得到相应时刻的流场流动数据。CAE-Transformer流动降阶模型的构造依赖于CAE网络的数据降维和Transformer网络的时间演化。与现有的POD/DMD等方法相比，使用CAE网络对流场数据进行非线性降维，同时使用Transformer网络对自由变量进行无方程演化，可以在保证流场降阶模型具备一定精度的情况下，得到更高的压缩比，提高流场预测的效率。

+ 输入：输入一段时间的流场。
+ 压缩：通过CAE的编码器对流场进行降维，提取高维时空流动特征。
+ 演化：通过Transformer学习低维空间流场时空特征的演变，预测下一时刻。
+ 重建：通过CAE的解码器将预测的流场低维特征恢复到高维空间。
+ 输出：输出对下一时刻瞬态流场的预测结果。

训练时，首先进行CAE网络的训练，训练完成之后使用CAE的编码器得到流场的低维特征，将此低维特征作为LSTM网络的数据集，进行LSTM网络的训练。

![CAE_Transformer1.png](./images/CAE_Transformer1.png)

## 训练环境

导入训练所需函数库，其中src包括数据集创建函数、网络模型和训练loss可视化函数。

训练默认采用Mindspore框架的静态图模式(GRAPH)，在GPU(默认)或Ascend进行训练(单卡)。

In [1]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindflow.utils import load_yaml_config

from src import create_cae_dataset, CaeNet, plot_train_loss

In [2]:
np.random.seed(0)
set_seed(0)

In [3]:
parser = argparse.ArgumentParser(description='cae net for kh')
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

## CAE网络训练参数设置

从config.yaml文件里导入数据集、CAE模型和优化器的参数配置。

In [4]:
config = load_yaml_config(args.config_file_path)
data_params = config["cae_data"]
model_params = config["cae_model"]
optimizer_params = config["cae_optimizer"]

训练过程loss文件保存路径默认为optimizer_params["summary_dir"]，权重参数保存在ckpt文件夹中。

In [5]:
summary_dir = optimizer_params["summary_dir"]
if not os.path.exists(summary_dir):
    os.mkdir(summary_dir)
ckpt_dir = os.path.join(summary_dir, 'ckpt')
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## 构建CAE网络

CAE网络由六层卷积和极大池化构成编码器，由七层卷积和上采样构成解码器。使用MSELoss损失函数和Adam优化器。

In [6]:
cae = CaeNet(model_params["data_dimension"], model_params["conv_kernel_size"], model_params["maxpool_kernel_size"],
             model_params["maxpool_stride"], model_params["encoder_channels"], model_params["decoder_channels"],
             model_params["channels_dense"])

loss_fn = nn.MSELoss()
cae_opt = nn.Adam(cae.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])

if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(cae, 'O1')
else:
    loss_scaler = None

## CAE网络数据集

数据集下载地址：data_driven/cae-lstm/kh/dataset

本案例数据集为二维开尔文-亥姆霍兹不稳定性问题的数值模拟结果，平行剪切流中的不稳定性叫做开尔文-亥姆霍兹不稳定性，坐标x和y范围为\[-0.5, 0.5\]，时间t范围为\[0, 1.5\]。共1786张流场快照，每张快照矩阵尺寸为(256, 256)。

导入数据集之后进行数据下沉设置。

In [7]:
cae_dataset, _ = create_cae_dataset(data_params["data_path"], data_params["batch_size"])

sink_process = data_sink(train_step, cae_dataset, sink_size=1)
train_data_size = cae_dataset.get_dataset_size()

## CAE网络模型训练

搭建forward_fn和train_step，开始CAE网络的训练，并将训练loss可视化。

In [8]:
def forward_fn(data, label):
    logits = cae(data)
    loss = loss_fn(logits, label)
    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, cae_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, cae_opt(grads))
    return loss

print(f"====================Start cae train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    cae.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(cae, f"{ckpt_dir}/cae_{epoch}.ckpt")
print(f"=====================End cae train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "cae")

pid:14003
====================Start cae train=======================
epoch: 1 train loss: 0.16278297 epoch time: 16.27s
epoch: 2 train loss: 0.098297514 epoch time: 2.26s
epoch: 3 train loss: 0.07453717 epoch time: 2.26s
epoch: 4 train loss: 0.05944114 epoch time: 2.27s
epoch: 5 train loss: 0.050014462 epoch time: 2.25s
...
epoch: 4396 train loss: 0.0009265681 epoch time: 2.23s
epoch: 4397 train loss: 0.00071872084 epoch time: 2.22s
epoch: 4398 train loss: 0.00075864815 epoch time: 2.22s
epoch: 4399 train loss: 0.00073658983 epoch time: 2.22
epoch: 4400 train loss: 0.0006535899 epoch time: 2.22s
====================End cae train=======================


## CAE流场重建结果

在训练完CAE网络后，可运行cae_prediction.py查看CAE的训练结果，以判断是否继续进行Transformer网络的训练。

下图分别为真实流场，CAE流场重建结果以及它们之间的误差曲线。其中前两个流场结果展现了流场中不同位置的密度随时间的变化情况，第三个误差曲线展现了CAE重建流场和真实流场label的平均相对误差随时间的变化情况。误差在大多数时间维持在0.015以下，满足流场重建精度需求。

<figure class="harf">
    <img src="./images/true.gif" title="cae_train_loss" width="300"/>
    <img src="./images/cae.gif" title="cae_prediction" width="300"/>
    <img src="./images/cae_error.png" title="cae_error" width="300"/>
</center>

## Transformer训练环境
导入训练所需函数库。

In [5]:
import numpy as np
import mindspore as ms

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindspore.train import CheckpointConfig, ModelCheckpoint
from mindvision.engine.callback import LossMonitor

from mindflow.utils import load_yaml_config
from sympy import Q
from src import create_transformer_dataset, plot_train_loss, Informer
from cae_prediction import cae_prediction

In [6]:
np.random.seed(42)
set_seed(42)

In [ ]:
parser = argparse.ArgumentParser(description="transformer net for KH")
parser.add_argument(
    "--mode",
    type=str,
    default="GRAPH",
    choices=["GRAPH", "PYNATIVE"],
    help="Context mode, support 'GRAPH', 'PYNATIVE'",
)
parser.add_argument(
    "--save_graphs",
    type=bool,
    default=False,
    choices=[True, False],
    help="Whether to save intermediate compilation graphs",
)
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument(
    "--device_target",
    type=str,
    default="GPU",
    choices=["GPU", "Ascend"],
    help="The target device to run, support 'Ascend', 'GPU'",
)
parser.add_argument(
    "--device_id", type=int, default=0, help="ID of the target device"
)
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

ms.set_context(device_target="GPU")
ms.set_context(mode=ms.PYNATIVE_MODE)
use_ascend = context.get_context(attr_key="device_target") == "Ascend"

## Transformer损失函数设置

CustomWithLossCell是一个在 MindSpore 中自定义的损失计算处理损失函数。

In [ ]:
class CustomWithLossCell(nn.Cell):
    def __init__(self, backbone, loss_fn, args):
        super(CustomWithLossCell, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn
        self.args = args

    def construct(self, seq_x, seq_y):
        cast = ops.Cast()

        batch_x = cast(seq_x, ms.float32)
        batch_y = cast(seq_y, ms.float32)

        if self.args["padding"] == 0:
            dec_inp = ops.Zeros()(
                (batch_y.shape[0], self.args["pred_len"], batch_y.shape[-1]), ms.float32
            )
        else:
            dec_inp = ops.Ones()(
                (batch_y.shape[0], self.args["pred_len"], batch_y.shape[-1]), ms.float32
            )
        dec_inp = cast(
            ops.concat([batch_x[:, - self.args["label_len"] : , :], dec_inp], axis=1),
            ms.float32,
        )

        outputs = self._backbone(batch_x, dec_inp)
        batch_y = batch_y[:, -self.args["pred_len"] :, :]

        return self._loss_fn(outputs, batch_y)

## Transformer网络框架及训练设置

首先导入Transformer模型数据集设置参数、Transformer模型和优化器参数设置。默认训练loss保存路径为optimizer_params["summary_dir"]，权重参数保存在ckpt文件夹下。模型由多个编码器和解码器堆叠而成，使用MSELoss损失函数和Adam优化器。

In [9]:
# prepare params
config = load_yaml_config(args.config_file_path)
data_params = config["transformer_data"]
model_params = config["transformer"]
optimizer_params = config["transformer_optimizer"]

# prepare summary file
summary_dir = optimizer_params["summary_dir"]
ckpt_dir = os.path.join(summary_dir, "ckpt")

# prepare model
model = Informer(**model_params)
loss_fn = nn.MSELoss()
optimizer = nn.Adam(
    model.trainable_params(),
    optimizer_params["lr"],
    weight_decay=optimizer_params["weight_decay"],
)


## Transformer网络数据集加载与处理

Transformer网络数据集由CAE的编码器得到，创建数据集。

In [10]:
# prepare dataset
latent_true = cae_prediction(args.config_file_path)
dataset, _ = create_transformer_dataset(
    latent_true,
    data_params["batch_size"],
    data_params["time_size"],
    data_params["latent_size"],
    data_params["time_window"],
    data_params["gaussian_filter_sigma"],
)

## Transformer网络模型训练

损失函数网络的构建、模型检查点回调函数的创建和trainer的初始化，开始Transformer网络的训练，并将训练loss可视化。

In [ ]:
time_now = time.time()
loss_net = CustomWithLossCell(model, loss_fn, data_params)
config = CheckpointConfig(save_checkpoint_steps=50, keep_checkpoint_max=2)
ckpt_callback = ModelCheckpoint(
     prefix="Informer", directory=ckpt_dir, config=config
)
trainer = ms.Model(network=loss_net, optimizer=optimizer)
for epoch in range(optimizer_params["epochs"]):
    print(f">>>>>>>>>>>>>>>>>>>>Train_{epoch}<<<<<<<<<<<<<<<<<<<<")
    ts = time.time()
    trainer.train(
        1,
        dataset,
        callbacks=[
            ckpt_callback,
            LossMonitor(optimizer_params["lr"], per_print_times=50),
        ],
    )
    print("Train Time Cost:", time.time() - ts)

## 预测流场结果可视化

运行cae_transformer_prediction.py可查看CAE-Transfomer降阶模型的预测结果。

下图分别为真实流场，CAE-Transformer网络的预测结果和相对应的平均相对误差。CAE-Transformer预测结果的误差比CAE重建的误差较大，是因为前者比后者多了Transformer的演化误差，但整个预测时间误差保持在0.02以下，满足流场预测精度需求。

<figure class="harf">
    <img src="./images/true2.gif" title="cae_prediction" width="300"/>
    <img src="./images/cae_lstm.gif" title="cae_lstm_prediction" width="300"/>
    <img src="./images/cae_lstm_error.png" title="cae_lstm_error" width="300"/>
</center>